<a href="https://colab.research.google.com/github/tlokeshkumar1/nlp/blob/master/third.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=4399d378d98e455eac9b658ebd3b77c52223034fe8ea71ff55d1eb9d074be1d5
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
#S-BERT-KG model for tweet classification
import pandas as pd
import numpy as np
import re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.spatial.distance import cosine

# Read the two CSV files and merge them into a single data frame
RESULTS_DIR = 'drive/MyDrive'
df1 = pd.read_csv(RESULTS_DIR + '/D11.csv', encoding='latin1', engine='python')
df2 = pd.read_csv(RESULTS_DIR + '/D2.csv', encoding='latin1', engine='python')
df = pd.concat([df1, df2], ignore_index=True)

# Extract the string attributes of hashtags using regular expressions
hashtags = df['hashtags'].apply(lambda x: re.findall(r'#\w+', x)).tolist()

# Define the set of tweets and labels
Advice = ["['Stay at home']","['wash hands']","['wear mask']","['social distancing']"]
China = ["['Wuhan']","['China Coronavirus Updates']","['China news']","['other tweets related to China']"]
Mask = ["['Mask shortage']","['wear mask']","['mask types']","['N50']","['N95']","['3M8210']","['3M9001']","['3M9322']","['3M9501']"]
News = ["['Coronavirus updates']","['news']","['rules']"]
Transportation = ["['Flights']","['traffic']","['traveling']"]
USA = ["['U.S. Coronavirus Updates']","['COVID19']","['U.S. news']","['United States']","['US']","['USA']"]
Vaccine = ["['Vaccine news']","['vaccine progress']","['vaccine injection']"]
tweets = Advice + China + Mask + News + Transportation + USA + Vaccine
labels = ["Vaccine","USA","Transportation","News","Mask","China","Advice"]

# Load a pre-trained S-BERT model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens', num_labels=len(labels))

# Embed the tweets and labels using the S-BERT model
tweet_embeddings = []
for tweet in tweets:
    encoded_tweet = tokenizer.encode(tweet, add_special_tokens=True, truncation=True, max_length=128, padding='max_length', return_tensors='pt')
    with torch.no_grad():
        tweet_embedding = model.bert(encoded_tweet)[1].numpy()
    tweet_embeddings.append(tweet_embedding)
tweet_embeddings = np.array(tweet_embeddings)

label_embeddings = []
for label in labels:
    encoded_label = tokenizer.encode(label, add_special_tokens=True, truncation=True, max_length=128, padding='max_length', return_tensors='pt')
    with torch.no_grad():
        label_embedding = model.bert(encoded_label)[1].numpy()
    label_embeddings.append(label_embedding)
label_embeddings = np.array(label_embeddings)

# Classify the tweets based on their cosine similarity to the label embeddings
for i, tweet in enumerate(df['hashtags']):
    encoded_tweet = tokenizer.encode(tweet, add_special_tokens=True, truncation=True, max_length=128, padding='max_length', return_tensors='pt')
    with torch.no_grad():
        tweet_embedding = model.bert(encoded_tweet)[1].numpy()
    similarities = [1 - cosine(np.squeeze(tweet_embedding), np.squeeze(label_embedding)) for label_embedding in label_embeddings]
    label_index = np.argmax(similarities)
    label = labels[label_index]
    print(f"Tweet {i+1}: {tweet}\nLabel: {label}\n")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/bert-base-nli-mean-tokens and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Label: Vaccine

Tweet 2113: ['China news']  
Label: China

Tweet 2114: ['rules']
Label: Transportation

Tweet 2115: ['United States']
Label: Transportation

Tweet 2116: ['3M8210']  
Label: News

Tweet 2117: ['United States']
Label: Transportation

Tweet 2118: ['3M9501']
Label: Vaccine

Tweet 2119: ['vaccine progress']  
Label: Vaccine

Tweet 2120: ['wash hands']   
Label: Transportation

Tweet 2121: ['Coronavirus updates']     
Label: Advice

Tweet 2122: ['Stay at home']  
Label: News

Tweet 2123: ['China news']  
Label: China

Tweet 2124: ['China news']  
Label: China

Tweet 2125: ['3M9001']
Label: News

Tweet 2126: ['United States']
Label: Transportation

Tweet 2127: ['Flights']  
Label: Vaccine

Tweet 2128: ['wear mask']
Label: Vaccine

Tweet 2129: ['wear mask']
Label: Vaccine

Tweet 2130: ['Wuhan']  
Label: Vaccine

Tweet 2131: ['wear mask']
Label: Vaccine

Tweet 2132: ['China news']  
Label: China

Tweet 2133: ['3M9501']
Label: Va

In [ ]:
pip install retrofitting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement retrofitting (from versions: none)
ERROR: No matching distribution found for retrofitting


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Read the CSV files containing the tweets
RESULTS_DIR = 'drive/MyDrive'
df1 = pd.read_csv(RESULTS_DIR + '/D11.csv', encoding='latin1', engine='python')
df2 = pd.read_csv(RESULTS_DIR + '/D2.csv', encoding='latin1', engine='python')
df = pd.concat([df1, df2], ignore_index=True)

# Extract the hashtags from the tweets
hashtags = df['hashtags'].tolist()

# Define the set of tweets and labels
tweets = hashtags
labels = ["Vaccine","USA","Transportation","News","Mask","China","Advice"]

# Define the set of keywords for each label
keywords = {
    "Vaccine": ["vaccine", "injection", "progress"],
    "USA": ["USA", "US", "United States", "COVID19"],
    "Transportation": ["flights", "traffic", "traveling"],
    "News": ["news", "updates", "rules"],
    "Mask": ["mask", "N50", "N95", "3M8210", "3M9001", "3M9322", "3M9501"],
    "China": ["Wuhan", "China", "Coronavirus Updates"],
    "Advice": ["stay at home", "wash hands", "wear mask", "social distancing"]
}

# Load the pre-trained S-BERT model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# Embed the tweets and labels into the S-BERT embedding space
tweet_embeddings = np.array([model(**tokenizer.encode_plus(tweet, return_tensors='pt')).last_hidden_state.detach().numpy()[0] for tweet in tweets])
label_embeddings = np.array([model(**tokenizer.encode_plus(' '.join(keywords[label]), return_tensors='pt')).last_hidden_state.detach().numpy()[0] for label in labels])

# Construct a knowledge graph embedding for all possible labels
knowledge_graph_embedding = np.mean(label_embeddings, axis=0)

# Learn a projection matrix to align the tweet and label embeddings
top_k_words = tokenizer.get_vocab().keys()[:10000]
word_embeddings = np.array([model(**tokenizer.encode_plus(word, return_tensors='pt')).last_hidden_state.detach().numpy()[0] for word in top_k_words])
projection_matrix = np.linalg.lstsq(word_embeddings, knowledge_graph_embedding, rcond=None)[0]

# Classify a given tweet by measuring the cosine similarity between its embedding and the embedding of each label
def classify_tweet(tweet):
    tweet_embedding = np.array(model(**tokenizer.encode_plus(tweet, return_tensors='pt')).last_hidden_state.detach().numpy()[0])
    projected_tweet_embedding = np.matmul(tweet_embedding, projection_matrix)
    similarities = np.dot(projected_tweet_embedding, label_embeddings.T) / (np.linalg.norm(projected_tweet_embedding) * np.linalg.norm(label_embeddings, axis=1))
    return labels[np.argmax(similarities)]

# Test the classification function on a sample hashtag

predicted_label = classify_tweet(hashtags)
print(predicted_label)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the pre-trained S-BERT model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/sentence-transformers-all-distilroberta-v1")
model = AutoModel.from_pretrained("sentence-transformers/sentence-transformers-all-distilroberta-v1")

# Load the ConceptNet knowledge graph
\

# (You will need to download and preprocess the ConceptNet graph)
graph = load_conceptnet_graph()

# Retrofit the ConceptNet graph to include sentence and label embeddings
graph = retrofit_graph(graph, model)

# Define the set of possible labels
labels = ["Vaccine","USA","Transportation","News","Mask","China","Advice"]

# Convert the labels to embeddings
label_embeddings = []
for label in labels:
    label_embedding = model.encode(label)
    label_embeddings.append(label_embedding)

# Loop through each tweet in X
for tweet in X:
    # Convert the tweet to an embedding
    tweet_embedding = model.encode(tweet)
    
    # Project the tweet and label embeddings to the knowledge graph embedding space
    projected_tweet_embedding = torch.matmul(tweet_embedding, P)
    projected_label_embeddings = []
    for label_embedding in label_embeddings:
        projected_label_embedding = torch.matmul(label_embedding, P)
        projected_label_embeddings.append(projected_label_embedding)
        
    # Calculate the cosine similarity between the tweet embedding and each label embedding
    similarities = []
    for projected_label_embedding in projected_label_embeddings:
        similarity = torch.cosine_similarity(projected_tweet_embedding, projected_label_embedding, dim=0)
        similarities.append(similarity)
        
    # Find the label with the highest similarity to the tweet
    max_similarity = max(similarities)
    max_similarity_index = similarities.index(max_similarity)
    predicted_label = labels[max_similarity_index]
    
    # Output the predicted label for the tweet
    print("Tweet: ", tweet)
    print("Predicted label: ", predicted_label)